## 제주 공공 와이파이 데이터 분석 (동부)

동부 : 구제주, 제주시 조천읍, 구좌읍, 우도면, 서귀포시 성산읍

In [1]:
import numpy as np 
import pandas as pd 

### 1. 버스(이동형) 와이파이 제외한 공공 와이파이 데이터 추출

wifi 접속 데이터

In [2]:
log_ = pd.read_csv('ap_acct_210410.csv')
log = log_[log_['ap_type']!='이동형']
fixed = pd.read_csv('fixed 2020.csv')
log.head(3)

,base_date,user_id,mac_address,upload,download,session_time,nationality,province,gender,age,visit_type,ap_type,ap_category,ap_place,ap_place_detail,ap_latitude,ap_longitude,ap_address_detail,pt_year,pt_month
1,20210410,0a5a417e7f5f88f4ee017504d6421c2083be1dec8f5cdf...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,282838,1230236,1198,대한민국,NaN,남성,40,거주,알수없음,NaN,NaN,NaN,NaN,NaN,NaN,2021,4
2,20210410,0a5a417e7f5f88f4ee017504d6421c2083be1dec8f5cdf...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,10859,8799,52,대한민국,NaN,남성,40,거주,알수없음,NaN,NaN,NaN,NaN,NaN,NaN,2021,4
3,20210410,d5ac5703133e9131c93b5033df85b3ab02fff9e93db95a...,e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b93...,70074,124090,1672,대한민국,NaN,남성,40,거주,알수없음,NaN,NaN,NaN,NaN,NaN,NaN,2021,4


wifi AP 데이터

In [3]:
fixed.head(3)

,base_date,manage_no,mac_address,ap_group_name,install_location_detail,open_date,category,category_detail,address_dong,address_detail,latitude,longitude,pt_year,pt_month
0,20200215,19-B0464,e3c2cffbac41cfd95b7acbec852d00aecfbdf7d5c059f0...,서귀포야구장,실내야구연습장,20191128,체육시설,체육시설,제주특별자치도 서귀포시 강정동,서귀포시 강정동 1552-3,33.260778,126.504462,2020,2
1,20200215,17-B0018,be5e5536d23a48a055c2f97b043e466a15340437476d7d...,엉또폭포,화장실 남,20170814,관광지,공영관광지,제주특별자치도 서귀포시 강정동,서귀포시 염돈로 121-6,33.265064,126.490320,2020,2
2,20200215,17-B0068,7807f89920f8154e7731f999c58fbc1124e2da3cd48286...,버스정류소_표선환승정류장(표선리사무소)[동],버스정류장 상부,20170827,버스정류소,버스정류소,제주특별자치도 서귀포시 표선면 표선리,서귀포시 표선면 표선리 631-1,33.325341,126.832510,2020,2


In [4]:
# 결측치 확인 
log.isna().sum()

base_date                 0
user_id                   0
mac_address               0
upload                    0
download                  0
session_time              0
nationality               0
province             211035
gender                    0
age                       0
visit_type                0
ap_type                   0
ap_category            1895
ap_place               1895
ap_place_detail        1895
ap_latitude            1895
ap_longitude           1895
ap_address_detail      1895
pt_year                   0
pt_month                  0
dtype: int64

In [5]:
fixed.isna().sum()

base_date                   0
manage_no                   0
mac_address                 0
ap_group_name               0
install_location_detail     2
open_date                   0
category                    0
category_detail            22
address_dong                4
address_detail              0
latitude                    0
longitude                   0
pt_year                     0
pt_month                    0
dtype: int64

### 2. 결측치 처리

총 226927개 중 211035개의 결측치를 갖는 log **province 열 삭제**

In [6]:
log = log.drop(['province'], axis=1)

총 226927개 중 1895개의 결측치를 갖는 **ap 관련 데이터 프레임 행 삭제**

In [7]:
# 1895개가 모두 동일한 데이터인지 확인 

list = ['ap_place', 'ap_place_detail', 'ap_latitude', 'ap_longitude', 'ap_address_detail']

for i in list:
    print(log[log['ap_category'].isna()==True][i].unique())
  

[nan]
[nan]
[nan]
[nan]
[nan]


In [8]:
list = ['ap_category', 'ap_place', 'ap_place_detail', 'ap_latitude', 'ap_longitude', 'ap_address_detail']
for i in list:
    log[i].fillna('알수없음', inplace=True)

### 3. 동부 지역별 데이터프레임화


In [9]:
# ap_address_detail(ap 주소) col 결측치 'X'로 대체
log['ap_address_detail'].fillna('X', inplace=True)

In [10]:
# 우도면
Udo = log[log['ap_address_detail'].str.contains('우도')]
# 조천읍
Jocheon = log[log['ap_address_detail'].str.contains('조천')]
# 구좌읍 
Gujwa = log[log['ap_address_detail'].str.contains('구좌')]
# 성산읍(서귀포까지)
Seongsan = log[log['ap_address_detail'].str.contains('성산')]

# 신제주(Gu_jeju)
engs = ['do', 'yong', 'one', 'two', 'hwa', 'sheep', 'ara', 'bong', 'three', 'gun']
kors = ['도두', '용담', '일도', '이도', '화북', '삼양', '아라', '봉개', '삼도', '건입']
list = []

for eng in engs:
    for kor in kors:
        eng = log[log['ap_address_detail'].str.contains(kor)]
        list.append(eng)

for i in range(1,10):
    list[0] = pd.concat([list[0], list[i]]) 

Gu_jeju = list[0]

### 4. 동일 관광지로 그룹화

In [11]:
# 보류

### 5. 같은 코스의 user_id > 같은 유형으로 간주

- have to be **exactly the same** so that you can consider those ids are the same travelling group
- but **except id who have under 3 course** for avoiding coincidence 

In [12]:
log['user_id'].unique()

array(['0a5a417e7f5f88f4ee017504d6421c2083be1dec8f5cdf4b499baead5d347c09',
       'd5ac5703133e9131c93b5033df85b3ab02fff9e93db95a17c2db3da42bc422d7',
       '94ca36d62db291c2962b0d89570589a17139f464eadb75819d276ba67a7c0aa6',
       ...,
       '56ae3b4f9e2bffd04e2a3e2e3a0ea3ba76ea84fca2dd817ab3841c2d7038bad5',
       '528c57070e2db24302ff2fbdb4805e08f1a612b77ebb5dc6aa14a95614888d62',
       '662c9e8cc6384c89482df0b2f02adee6d764fc59fec23b0abc66057a790b018e'],
      dtype=object)

In [13]:
course = (log[log['user_id']=='56ae3b4f9e2bffd04e2a3e2e3a0ea3ba76ea84fca2dd817ab3841c2d7038bad5']['ap_place'].unique())
course[0]

'전기차충전소_제주시청'

In [14]:
log['user_id'].unique()[0]

'0a5a417e7f5f88f4ee017504d6421c2083be1dec8f5cdf4b499baead5d347c09'

In [15]:
id = log['user_id'].unique().tolist()
list = []

for i in id:
    for j in range(14753):
        if len(log[log['user_id']== i]['ap_place'].unique()) > 3:
            list.append(log[log['user_id']== i])
        else :
            pass
        

KeyboardInterrupt: 

5. 이지인 이소의 양현진 

In [ ]:
log[log['ap_place']=='전기차충전소_제주시청']

,base_date,user_id,mac_address,upload,download,session_time,nationality,gender,age,visit_type,ap_type,ap_category,ap_place,ap_place_detail,ap_latitude,ap_longitude,ap_address_detail,pt_year,pt_month
41450,20210410,b5cd03f6fe1b0fa98316767d6a47a2977ff3bbd314c7f9...,5df3377d578b1fa525c48f0e99bb4445adba7189b9745b...,0,0,395,대한민국,여성,40,업무,고정형,전기차충전소,전기차충전소_제주시청,제주시청#4(4별관)-2,33.499321,126.530459,제주시 이도2동 1176-118 (동광로2길 7) 옆,2021,4
41451,20210410,fda9faa68e3d87a7ad505175da0d69335424b63f346046...,5df3377d578b1fa525c48f0e99bb4445adba7189b9745b...,2188,1146,86767,대한민국,여성,10,거주,고정형,전기차충전소,전기차충전소_제주시청,제주시청#4(4별관)-2,33.499321,126.530459,제주시 이도2동 1176-118 (동광로2길 7) 옆,2021,4
41452,20210410,7742bd77da456fc101b02ae075aa434f7cfc264554b5cf...,5df3377d578b1fa525c48f0e99bb4445adba7189b9745b...,2838,1096,86772,대한민국,여성,50,관광,고정형,전기차충전소,전기차충전소_제주시청,제주시청#4(4별관)-2,33.499321,126.530459,제주시 이도2동 1176-118 (동광로2길 7) 옆,2021,4
41453,20210410,967732b6e079e8eaca361c335ac55993f0d6f7d4ae9d01...,5df3377d578b1fa525c48f0e99bb4445adba7189b9745b...,0,100,86761,대한민국,남성,20,관광,고정형,전기차충전소,전기차충전소_제주시청,제주시청#4(4별관)-2,33.499321,126.530459,제주시 이도2동 1176-118 (동광로2길 7) 옆,2021,4
41454,20210410,c068926dc3270d33bd4574ddc5748e8d72ce71594e876d...,5df3377d578b1fa525c48f0e99bb4445adba7189b9745b...,34,428,86773,대한민국,여성,40,거주,고정형,전기차충전소,전기차충전소_제주시청,제주시청#4(4별관)-2,33.499321,126.530459,제주시 이도2동 1176-118 (동광로2길 7) 옆,2021,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
294019,20210410,9f2002f779584c0723715dee74c665b7d96b837dfc6dfa...,3000408bfe0709ed87994ea9b4a42e242c2a3e40027255...,156148,2222129,86811,대한민국,남성,10,거주,고정형,전기차충전소,전기차충전소_제주시청,제주시청 #2,33.500267,126.532656,제주시 이도 2동 1176-1,2021,4
294020,20210410,badb85d2b29b752e13505bfb63b7c5046333af03a4769d...,3000408bfe0709ed87994ea9b4a42e242c2a3e40027255...,876,996,12,대한민국,여성,20,거주,고정형,전기차충전소,전기차충전소_제주시청,제주시청 #2,33.500267,126.532656,제주시 이도 2동 1176-1,2021,4
294021,20210410,938a6c745fb9253ffecb97dde87e46e7e9940d0df2985a...,3000408bfe0709ed87994ea9b4a42e242c2a3e40027255...,0,0,365,대한민국,남성,40,기타,고정형,전기차충전소,전기차충전소_제주시청,제주시청 #2,33.500267,126.532656,제주시 이도 2동 1176-1,2021,4
294022,20210410,14a64586c2a852a1c924fdd89a5de23cf1875c49510c2a...,3000408bfe0709ed87994ea9b4a42e242c2a3e40027255...,63065,121941,643,대한민국,남성,50,업무,고정형,전기차충전소,전기차충전소_제주시청,제주시청 #2,33.500267,126.532656,제주시 이도 2동 1176-1,2021,4


In [ ]:
user = log['user_id'].unique().tolist()

for i in user:
    if (log[log['user_id']==i]['ap_place'].unique()) == course :
        print(i) 
    else :
        print('no')

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()